In [ ]:
# Setup on Colab
! pip install transformers evaluate torch rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/AI/VJ/mt5-work/data/articles_and_summaries.csv'
df = pd.read_csv(file_path)

df_info = df.info()
df_head = df.head()

df_info, df_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5320 entries, 0 to 5319
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Article         5320 non-null   object
 1   Summary         5320 non-null   object
 2   Article Length  5320 non-null   int64 
 3   Summary Length  5320 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 166.4+ KB


(None,
                                              Article  \
 0  第一条 私権は、公共の福祉に適合しなければならない。権利の行使及び義務の履行は、信義に従い誠...   
 1         第二条 この法律は、個人の尊厳と両性の本質的平等を旨として、解釈しなければならない。   
 2  第三条 私権の享有は、出生に始まる。外国人は、法令又は条約の規定により禁止される場合を除き、...   
 3  第三条の二 法律行為の当事者が意思表示をした時に意思能力を有しなかったときは、その法律行為は...   
 4                               第四条 年齢十八歳をもって、成年とする。   
 
                                              Summary  Article Length  \
 0  私権は公共の福祉に適し、誠実に信義に従って行使・履行されなければならない。権利の濫用は認めら...              74   
 1                         個人の尊厳と男女の平等を原則とした法律の解釈が必要。              42   
 2        第3条 出生から私権を有する。外国人も、法律や条約で禁止されていなければ私権を有する。              54   
 3                          意思能力がない時に法律行為をすると無効になります。              53   
 4                                           成人年齢は18歳              20   
 
    Summary Length  
 0              50  
 1              26  
 2              43  
 3              25  
 4               8  )

In [ ]:
max_summary_length = df['Summary Length'].max()
max_article_length = df['Article Length'].max()

max_summary_length, max_article_length

(411, 6064)

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from transformers import MT5ForConditionalGeneration, T5Tokenizer, AdamW
import evaluate
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

In [ ]:
class TextSummarizationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=1024):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        text = row['Article']
        summary = row['Summary']
        inputs = self.tokenizer(text, max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt")
        targets = self.tokenizer(summary, max_length=256, truncation=True, padding="max_length", return_tensors="pt")
        return {
            'input_ids': inputs.input_ids.squeeze(0),
            'attention_mask': inputs.attention_mask.squeeze(0),
            'labels': targets.input_ids.squeeze(0)
        }

In [ ]:
class MT5SUM_FT:
    def __init__(self, model_name, tokenizer_name, device='cuda', max_length=1024, show_tqdm=True):
        self.tokenizer = T5Tokenizer.from_pretrained(tokenizer_name)
        self.model = MT5ForConditionalGeneration.from_pretrained(model_name)
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        self.max_length = max_length
        self.rouge = evaluate.load("rouge")
        self.show_tqdm = show_tqdm

    def prepare_data(self, filepath):
        df = pd.read_csv(filepath)
        train_df, val_df = train_test_split(df, test_size=0.1)
        train_dataset = TextSummarizationDataset(train_df, self.tokenizer, self.max_length)
        val_dataset = TextSummarizationDataset(val_df, self.tokenizer, self.max_length)
        return DataLoader(train_dataset, batch_size=4, shuffle=True), DataLoader(val_dataset, batch_size=4)

    def train(self, train_dataloader, val_dataloader, epochs=3, lr=5e-5):
        optimizer = AdamW(self.model.parameters(), lr=lr)
        for epoch in range(epochs):
            self.model.train()
            total_train_loss = 0
            train_iterator = tqdm(train_dataloader, desc=f"Epoch {epoch+1} Training", disable=not self.show_tqdm)
            for batch in train_iterator:
                batch = {k: v.to(self.device) for k, v in batch.items()}
                outputs = self.model(**batch)
                loss = outputs.loss
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                total_train_loss += loss.item()

            self.model.eval()
            total_val_loss = 0
            rouge_scores = {key: [] for key in ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']}
            val_iterator = tqdm(val_dataloader, desc=f"Epoch {epoch+1} Validation", disable=not self.show_tqdm)
            for batch in val_iterator:
                batch = {k: v.to(self.device) for k, v in batch.items()}
                with torch.no_grad():
                    outputs = self.model(**batch)
                    loss = outputs.loss
                    total_val_loss += loss.item()

                    predicted_summaries = self.tokenizer.batch_decode(outputs.logits.argmax(dim=-1), skip_special_tokens=True)
                    references = [self.tokenizer.decode(ids, skip_special_tokens=True) for ids in batch['labels']]
                    batch_scores = self.rouge.compute(predictions=predicted_summaries, references=references)

                    # print("Batch scores:", batch_scores)

                    for key in rouge_scores.keys():
                        rouge_scores[key].append(batch_scores[key])

            avg_train_loss = total_train_loss / len(train_dataloader)
            avg_val_loss = total_val_loss / len(val_dataloader)
            final_rouge_scores = {key: sum(vals)/len(vals) for key, vals in rouge_scores.items()}
            print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss}, Val Loss = {avg_val_loss}, ROUGE Scores = {final_rouge_scores}")

    def save_model(self, path):
        self.model.save_pretrained(path)
        self.tokenizer.save_pretrained(path)

    def predict(self, text, model_path):
        self.model = MT5ForConditionalGeneration.from_pretrained(model_path).to(self.device)
        self.tokenizer = T5Tokenizer.from_pretrained(model_path)
        input_ids = self.tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=self.max_length, truncation=True).to(self.device)
        outputs = self.model.generate(input_ids, max_length=256, num_beams=15, early_stopping=True)
        summary = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return summary


In [ ]:
if __name__ == "__main__":
        model_name = 'tsmatz/mt5_summarize_japanese'
        tokenizer_name = 'tsmatz/mt5_summarize_japanese'
        device = 'cuda'
        csv_file_path = '/content/drive/MyDrive/AI/VJ/mt5-work/data/articles_and_summaries.csv'
        model_save_path = '/content/drive/MyDrive/AI/VJ/mt5-work/models/improved_mt5-summarize'
        max_length = 1024
        epochs = 10
        batch_size =16
        learning_rate = 5e-5
        show_tqdm = True

        finetuner = MT5SUM_FT(model_name, tokenizer_name, device, max_length, show_tqdm)
        train_loader, val_loader = finetuner.prepare_data(csv_file_path)
        finetuner.train(train_loader, val_loader, epochs, learning_rate)
        finetuner.save_model(model_save_path)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1 Training:   0%|          | 0/1197 [00:00<?, ?it/s]

Epoch 1 Validation:   0%|          | 0/133 [00:00<?, ?it/s]

Epoch 1: Train Loss = 2.0776150044841177, Val Loss = 0.5209051288832399, ROUGE Scores = {'rouge1': 0.14105132299797304, 'rouge2': 0.0353066647892696, 'rougeL': 0.1373989133775183, 'rougeLsum': 0.1373989133775183}


Epoch 2 Training:   0%|          | 0/1197 [00:00<?, ?it/s]

...

Epoch 9: Train Loss = 0.37404211844592267, Val Loss = 0.34316764469433547, ROUGE Scores = {'rouge1': 0.27579709512384226, 'rouge2': 0.11422024576389969, 'rougeL': 0.27274698957196625, 'rougeLsum': 0.27274698957196625}


Epoch 10 Training:   0%|          | 0/1197 [00:00<?, ?it/s]

Epoch 10 Validation:   0%|          | 0/133 [00:00<?, ?it/s]

Epoch 10: Train Loss = 0.363207462543284, Val Loss = 0.3387773635244011, ROUGE Scores = {'rouge1': 0.2857976230842446, 'rouge2': 0.12418960481733377, 'rougeL': 0.283530354884399, 'rougeLsum': 0.28346769824279494}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'num_beams': 15, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}


In [ ]:
sample_text = "第二十七条 前二条の規定により家庭裁判所が選任した管理人は、その管理すべき財産の目録を作成しなければならない。この場合において、その費用は、不在者の財産の中から支弁する。不在者の生死が明らかでない場合において、利害関係人又は検察官の請求があるときは、家庭裁判所は、不在者が置いた管理人にも、前項の目録の作成を命ずることができる。前二項に定めるもののほか、家庭裁判所は、管理人に対し、不在者の財産の保存に必要と認める処分を命ずることができる。"
print("Original Text:", sample_text)
print("\nGenerated Summary:", finetuner.predict(sample_text, model_save_path))

Original Text: 第二十七条 前二条の規定により家庭裁判所が選任した管理人は、その管理すべき財産の目録を作成しなければならない。この場合において、その費用は、不在者の財産の中から支弁する。不在者の生死が明らかでない場合において、利害関係人又は検察官の請求があるときは、家庭裁判所は、不在者が置いた管理人にも、前項の目録の作成を命ずることができる。前二項に定めるもののほか、家庭裁判所は、管理人に対し、不在者の財産の保存に必要と認める処分を命ずることができる。

Generated Summary: 家庭裁判所が選任した管理人は、管理すべき財産の目録を作成する必要があります。費用は不在者の財産から支弁されます。利害関係人や検察官請求がある場合は、管理人に処分を命じることができます。


In [ ]:
# Chat GPT Gen Sample_text
sample_text = "第四条　個人情報取扱事業者は、取得した個人情報を、あらかじめ公表した利用目的の範囲内でのみ使用し、目的外利用を行わないように適切な措置を講じるものとする。"
print("Original Text:", sample_text)
print("\nGenerated Summary:", finetuner.predict(sample_text, model_save_path))

Original Text: 第四条　個人情報取扱事業者は、取得した個人情報を、あらかじめ公表した利用目的の範囲内でのみ使用し、目的外利用を行わないように適切な措置を講じるものとする。

Generated Summary: 個人情報取扱事業者は、利用目的の範囲内でのみ使用し、目的外利用を行わないよう適切な措置を講じる。


In [ ]:
# This result keeps the number of years :D
sample_text = "各当事者は、契約が有効である間及び契約終了後少なくとも三（3）年間、相手方から受け取ったすべての情報を秘密にすることを約束します。秘密情報には、個人データ、財務情報、ビジネス戦略、製品またはサービスの詳細が含まれますが、これに限定されません。"
print("Original Text:", sample_text)
print("\nGenerated Summary:", finetuner.predict(sample_text, model_save_path))

Original Text: 各当事者は、契約が有効である間及び契約終了後少なくとも三（3）年間、相手方から受け取ったすべての情報を秘密にすることを約束します。秘密情報には、個人データ、財務情報、ビジネス戦略、製品またはサービスの詳細が含まれますが、これに限定されません。

Generated Summary: 契約有効間と契約終了後3年間、相手方から受け取った情報を秘密に約束します。秘密情報には個人データ、財務情報、サービスの詳細が含まれます。
